In [1]:
import plotly.express as px
from pyspark.sql.functions import *

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('flights').getOrCreate()

23/01/02 20:49:20 WARN Utils: Your hostname, MacBook-Air-di-Teodoro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.156 instead (on interface en0)
23/01/02 20:49:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/02 20:49:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv('../../data.nosync/',inferSchema=True,header=True)

In [8]:
df_prep = df.groupBy('FlightDate').agg({'Diverted':'sum',"Cancelled":'sum'})
df_prep = df_prep.withColumnRenamed('sum(Diverted)','Diverted')
df_prep = df_prep.withColumnRenamed('sum(Cancelled)','Cancelled')

In [9]:
df_prep.show()

+-------------------+--------+---------+
|         FlightDate|Diverted|Cancelled|
+-------------------+--------+---------+
|2013-06-12 00:00:00|   123.0|    597.0|
|2013-06-13 00:00:00|    97.0|    719.0|
|2013-06-05 00:00:00|    56.0|    203.0|
|2013-06-24 00:00:00|    64.0|    656.0|
|2013-06-10 00:00:00|    60.0|    363.0|
|2013-06-19 00:00:00|    21.0|    166.0|
|2013-06-06 00:00:00|    74.0|    231.0|
|2013-06-15 00:00:00|    15.0|    110.0|
|2013-06-01 00:00:00|    49.0|    135.0|
|2013-06-26 00:00:00|    94.0|    648.0|
|2013-06-29 00:00:00|    34.0|    347.0|
|2013-06-02 00:00:00|    65.0|    398.0|
|2013-06-18 00:00:00|    79.0|    540.0|
|2013-06-25 00:00:00|    54.0|    454.0|
|2013-06-30 00:00:00|    71.0|    350.0|
|2013-06-22 00:00:00|    62.0|    101.0|
|2013-06-20 00:00:00|    26.0|     93.0|
|2013-06-03 00:00:00|    30.0|    209.0|
|2013-06-23 00:00:00|    85.0|    229.0|
|2013-06-16 00:00:00|    26.0|    106.0|
+-------------------+--------+---------+
only showing top

In [10]:
# convert to pandas mantain flightdate as timestamp
df_prep = df_prep.toPandas()
df_prep['FlightDate'] = pd.to_datetime(df_prep['FlightDate'])


/opt/homebrew/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [12]:
df_prep

,FlightDate,Diverted,Cancelled
0,2013-06-12,123.0,597.0
1,2013-06-13,97.0,719.0
2,2013-06-05,56.0,203.0
3,2013-06-24,64.0,656.0
4,2013-06-10,60.0,363.0
...,...,...,...
360,2013-08-26,48.0,103.0
361,2013-08-24,17.0,50.0
362,2013-08-27,18.0,104.0
363,2013-08-11,25.0,112.0


In [11]:
# save df_pd to a csv file
df_pd.to_csv('../../util/cancelled_diverted.csv',index=False)

In [4]:
dates = df.select('FlightDate').distinct().orderBy('FlightDate',ascending=True).toPandas()['FlightDate']

/opt/homebrew/lib/python3.10/site-packages/pyspark/sql/pandas/conversion.py:248: FutureWarning: Passing unit-less datetime64 dtype to .astype is deprecated and will raise in a future version. Pass 'datetime64[ns]' instead
  series = series.astype(t, copy=False)


In [5]:
import pandas as pd
cancelled_diverted=pd.read_csv('../../util/cancelled_diverted.csv')

In [18]:
# get the type of FlightDate column in cancelled_diverted
type(cancelled_diverted['FlightDate'][0])
# convert to timestamp

cancelled_diverted

,FlightDate,Diverted,Cancelled
0,2013-06-12,123.0,597.0
1,2013-06-13,97.0,719.0
2,2013-06-05,56.0,203.0
3,2013-06-24,64.0,656.0
4,2013-06-10,60.0,363.0
...,...,...,...
360,2013-08-26,48.0,103.0
361,2013-08-24,17.0,50.0
362,2013-08-27,18.0,104.0
363,2013-08-11,25.0,112.0


In [16]:
cd_filtered = cancelled_diverted[(cancelled_diverted['FlightDate'] >= dates[0]) & (cancelled_diverted['FlightDate'] <= dates[100])]

In [17]:
len(cd_filtered)

101

In [6]:
def textual_queries(df,from_date,to_date):
    #df = df.filter(df["FlightDate"].between(from_date,to_date))
    # filter the pandas dataframe cancelled_diverted by the dates
    cd_filtered = cancelled_diverted[(cancelled_diverted['FlightDate'] >= from_date) & (cancelled_diverted['FlightDate'] <= to_date)]
    #'textual-num','textual-airports','textual-cancelled','textual-delayed','textual-diverted','textual-average-delay'

    num = df.count()
    airports = df.select('Origin').distinct().count()+df.select('Dest').distinct().count()
    canceled = cd_filtered['Cancelled'].sum()
    # count the rows where ArrDelay is > 0
    delayed = df.filter(df["ArrDelay"] > 0).count()

    diverted = cd_filtered['Diverted'].sum()
    # make the average of ArrDelay
    average_delay = df.agg({"ArrDelay": "avg"}).collect()[0][0]

    return num,airports,canceled,delayed,diverted,average_delay